In [ ]:
from suds.client import Client
from suds.xsd.doctor import ImportDoctor, Import

import pandas as pd
import numpy as np

 Функция, получающая показтели нормативов банка с данной лицензией на данную дату с сайта cbr.ru

In [ ]:
def load_cbr_standarts(license_num, date):
    # loads values of standarts (aka normativs) which are binding on russian banks
    # license_num -- int value
    # date -- string 'yyyy-mm-ddT00:00:00+03:00'
    #    e.g. July, 1 2016 --> '2016-07-01T00:00:00+03:00'
    # this data is available for the fist days of each month
    # returns pandas data frame with values of standarts for chosen bank & date
    url = 'http://www.cbr.ru/CreditInfoWebServ/CreditOrgInfo.asmx?WSDL'
    imp = Import('http://schemas.xmlsoap.org/soap/encoding/')
    imp.filter.add('http://web.cbr.ru/')
    imp = Import('http://www.w3.org/2001/XMLSchema', location='http://www.w3.org/2001/XMLSchema.xsd')
    imp.filter.add('http://web.cbr.ru/')
    doctor = ImportDoctor(imp)
    client = Client(url, doctor=doctor)
    tmp = client.service.Data135FormFull(license_num, date)
    try:
        norms = tmp.diffgram.F135DATA.F135_3
    except AttributeError:
        return -1
    df = pd.DataFrame()
    for i in range(len(norms)):
        ind_name = norms[i].C3
        try:
            ind_value = norms[i].V3
        except AttributeError:
            ind_value = None
        df_tmp = pd.DataFrame({'Indicator_name': [ind_name], 'Indicator_value': [ind_value]})
        df = df.append(df_tmp, ignore_index=True)
    return df


Немного форматируем дату

In [ ]:
def mts(month):
    if month<10:
        return '0'+str(month)
    else:
        return str(month)

Загружаем список лицензий

In [ ]:
files = list(pd.read_csv('data/files',header=None)[0].values)

Загружаем банки

In [ ]:
banks = {}
for i in files:
    banks[i] = pd.read_csv('data2/'+str(i)+'.csv',encoding='UTF-8')

Загружаем показатели нормативов для всех банков

In [ ]:
for j,name in enumerate(files):
    if j%10==0:
        print(j)
    sozd = 0
    for i in range(len(banks[name])):
        try:
            tmp = load_cbr_standarts(name, str(banks[name]['year'][i])+'-'+mts(banks[name]['month'][i])+'-01T00:00:00+03:00')
        except AttributeError:
            tmp = -1
        if type(tmp)!=int:
            tmp = tmp.set_index(tmp['Indicator_name'])
            del tmp['Indicator_name']
            tmp = tmp.transpose()
            tmp.index = [i]
            if sozd==0:
                values = tmp
                sozd = 1
            else:
                values = values.append(tmp)
    banks[name] = pd.concat([banks[name], values], axis=1)
    banks[name].to_csv('data2/'+str(name)+'.csv',index=False,encoding='UTF-8')

Для разных банков и дат получились разные списки доступных нормативов. Собираем все полученные нормативы в один массив

In [ ]:
t1 = [x for x in banks[files[0]][banks[files[0]].columns[59]] if str(x) != 'nan']
for i in files[1:]:
    t2 = [x for x in banks[i][banks[i].columns[59]] if str(x) != 'nan']
    if t1==t2:
        banks[i] = pd.read_csv('data/'+str(i)+'.csv')
        for j in banks[1].columns[58:]:
            banks[i][j] = np.nan
    t1 = t2

От норматива Н1 в какой-то момент перешли к нормативам Н1.0, Н1.1 и Н1.2. Соединяем эти показатели

In [ ]:
for k in files:
    if u'Н1' in banks[k].columns:
        for i in [u'Н1.0',u'Н1.1',u'Н1.2']:
            if i in banks[k].columns:
                banks[k][i][((banks[k]['year']-2008)*12+banks[k]['month'])<74] = banks[k][u'Н1'][((banks[k]['year']-2008)*12+banks[k]['month'])<74]

Опять считаем пропуски

In [ ]:
non = {}
for j in banks[1].columns:
    non[j] = 0
for i in files:
    for j in banks[1].columns:
        if j in banks[i].columns:
            non[j]+=np.sum(banks[i][((banks[i]['year']-2008)*12+banks[i]['month'])>37][j].isnull())
        else:
            non[j]+=len(banks[i][((banks[i]['year']-2008)*12+banks[i]['month'])>37])

Ищем "плохие" признаки с большим количество пропусков

In [ ]:
bad = []
for i in non.keys():
    if non[i]>10000:
        bad.append(i)
for i in files:
    for j in bad:
        if j in banks[i].columns:
            del banks[i][j]

Вычисляем максимальные и минимальные показатели для значений нормативов

In [ ]:
maxv = {}
minv = {}
for j in banks[1].columns[58:]:
    maxv[j] = np.max(banks[1][j])
    minv[j] = np.min(banks[1][j])
for i in files:
    for j in banks[1].columns[58:]:
        if j in banks[i].columns:
            maxv[j] = max(maxv[j],np.max(banks[i][j]))
            minv[j] = min(minv[j],np.min(banks[i][j]))

Назначаем значения для пропусков

In [ ]:
fillin = minv.copy()
fillin[u'\u041d1.0'] = -100
fillin[u'\u041d1.1'] = -100
fillin[u'\u041d1.2'] = -100
fillin[u'\u041d10.1'] = -100
fillin[u'\u041d12'] = -100
fillin[u'\u041d2'] = -10
fillin[u'\u041d3'] = -10
fillin[u'\u041d4'] = -10
fillin[u'\u041d7'] = -10000
fillin[u'\u041d9.1'] = -100

Заполняем пропуски (в отличие от парсинга сайта banki.ru мы более лояльны к пропускам, так как несоотвествие нормативам является важныи индикатором для скорого отзыва лицензии)

In [ ]:
for i in files:
    for j in banks[1].columns[58:]:
        if j in banks[i].columns:
            banks[i][j] = banks[i][j].fillna(value=fillin[j])
        else:
            banks[i][j] = fillin[j]

Созраняем изменения

In [ ]:
for i in files:
    banks[i].to_csv('data2/'+str(i)+'.csv',encoding='UTF-8',index=False)